In [1]:
!pip install transformers

### __一、处理数据:__

这一小节学习第一小节中提到的“如何使用模型中心（hub）大型数据集”，下面是我们用模型中心的数据在PyTorch上训练句子分类器的一个例子：

In [2]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
batch

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2023,  2607,  2003,  6429,   999,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([1, 1])}

In [4]:
optimizer

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    lr: 0.001
    weight_decay: 0.0
)

当然，仅仅用两句话训练模型不会产生很好的效果。为了获得更好的结果，您需要准备一个更大的数据集。

在本节中，我们将使用MRPC（微软研究释义语料库）数据集作为示例，该数据集由威廉·多兰和克里斯·布罗克特在这篇文章发布。该数据集由5801对句子组成，每个句子对带有一个标签，指示它们是否为同义（即，如果两个句子的意思相同）。我们在本章中选择了它，因为它是一个小数据集，所以很容易对它进行训练。

### __二、从模型中心（Hub）加载数据集__

模型中心（hub）不只是包含模型；它也有许多不同语言的多个[数据集](https://huggingface.co/datasets)。点击数据集的链接即可进行浏览。我们建议您在阅读本节后阅读一下[加载和处理新的数据集](https://huggingface.co/docs/datasets/loading_datasets.html#from-the-huggingface-hub)这篇文章，这会让您对huggingface的darasets更加清晰。但现在，让我们使用MRPC数据集中的 [GLUE 基准测试数据集](https://gluebenchmark.com/)，它是构成MRPC数据集的10个数据集之一，这是一个学术基准，用于衡量机器学习模型在10个不同文本分类任务中的性能。

🤗 Datasets库提供了一个非常便捷的命令，可以在模型中心（hub）上下载和缓存数据集。我们可以通过以下的代码下载MRPC数据集：

In [6]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

正如你所看到的，我们获得了一个DatasetDict对象，其中包含训练集、验证集和测试集。每一个集合都包含几个列(sentence1, sentence2, label, and idx)以及一个代表行数的变量，即每个集合中的行的个数（因此，训练集中有3668对句子，验证集中有408对，测试集中有1725对）。

默认情况下，此命令在下载数据集并缓存到 ~/.cache/huggingface/dataset. 可以通过设置HF_HOME环境变量来自定义缓存的文件夹。

我们可以访问我们数据集中的每一个raw_train_dataset对象，如使用字典：

In [7]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

我们可以看到标签已经是整数了，所以我们不需要对标签做任何预处理。要知道哪个数字对应于哪个标签，我们可以查看raw_train_dataset的features. 这将告诉我们每列的类型：

In [8]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

在上面的例子之中,Label（标签） 是一种ClassLabel（分类标签），使用整数建立起到类别标签的映射关系。0对应于not_equivalent，1对应于equivalent。

### __三、预处理数据集__

为了预处理数据集，我们需要将文本转换为模型能够理解的数字。

In [9]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

然而，在两句话传递给模型，预测这两句话是否是同义之前。我们需要这两句话依次进行适当的预处理。幸运的是，标记器不仅仅可以输入单个句子还可以输入一组句子，并按照我们的BERT模型所期望的输入进行处理：

In [10]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

我们在第二章 讨论了输入词id(input_ids) 和 注意力遮罩(attention_mask) ，但我们在那个时候没有讨论类型标记ID(token_type_ids)。在这个例子中，类型标记ID(token_type_ids)的作用就是告诉模型输入的哪一部分是第一句，哪一部分是第二句。

如果我们将input_ids中的id转换回文字:

In [11]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

所以我们看到模型需要输入的形式是 [CLS] sentence1 [SEP] sentence2 [SEP]。因此，当有两句话的时候。类型标记ID(token_type_ids) 的值是：

In [12]:
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[   0,      0,     0,     0,     0,          0,      0,     0,      1,    1,     1,        1,      1,    1,     1]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

如您所见，输入中 [CLS] sentence1 [SEP] 它们的类型标记ID均为0，而其他部分，对应于sentence2 [SEP]，所有的类型标记ID均为1.

请注意，如果选择其他的检查点，则不一定具有类型标记ID(token_type_ids)（例如，如果使用DistilBERT模型，就不会返回它们）。只有当它在预训练期间使用过这一层，模型在构建时依赖它们，才会返回它们。

用类型标记ID对BERT进行预训练,并且使用第一章的遮罩语言模型，还有一个额外的应用类型，叫做下一句预测. 这项任务的目标是建立成对句子之间关系的模型。

在下一个句子预测任务中，会给模型输入成对的句子（带有随机遮罩的标记），并被要求预测第二个句子是否紧跟第一个句子。为了提高模型的泛化能力，数据集中一半的两个句子在原始文档中挨在一起，另一半的两个句子来自两个不同的文档。

一般来说，你不需要担心是否有类型标记ID(token_type_ids)。在您的标输入中：只要您对标记器和模型使用相同的检查点，一切都会很好，因为标记器知道向其模型提供什么。

现在我们已经了解了标记器如何处理一对句子，我们可以使用它对整个数据集进行处理：如之前的章节，我们可以给标记器提供一组句子，第一个参数是它第一个句子的列表，第二个参数是第二个句子的列表。这也与我们在第二章中看到的填充和截断选项兼容. 因此，预处理训练数据集的一种方法是：

In [16]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

tokenized_dataset[0]

[Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [34]:
# tokenized_dataset

In [37]:
print(tokenized_dataset.input_ids[0])
print(tokenized_dataset.attention_mask[0])

[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


这很有效，但它的缺点是返回字典（字典的键是输入词id(input_ids) ， 注意力遮罩(attention_mask) 和 类型标记ID(token_type_ids)，字典的值是键所对应值的列表）。而且只有当您在转换过程中有足够的内存来存储整个数据集时才不会出错（而🤗数据集库中的数据集是以Apache Arrow文件存储在磁盘上，因此您只需将接下来要用的数据加载在内存中，因此会对内存容量的需求要低一些）。

为了将数据保存为数据集，我们将使用Dataset.map()方法，如果我们需要做更多的预处理而不仅仅是标记化，那么这也给了我们一些额外的自定义的方法。这个方法的工作原理是在数据集的每个元素上应用一个函数，因此让我们定义一个标记输入的函数：

In [30]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

<font color = '#EC7063'>此函数的输入是一个字典（与数据集的项类似），并返回一个包含输入词 id(input_ids) ， 注意力遮罩 (attention_mask) 和 类型标记 ID(token_type_ids) 键的新字典。

请注意，如果像上面的示例一样，如果键所对应的值包含多个句子（每个键作为一个句子列表），那么它依然可以工作，就像前面的例子一样标记器可以处理成对的句子列表。这样的话我们可以在调用map()使用该选项 batched=True ，这将显著加快标记与标记的速度。这个标记器来自🤗 Tokenizers库由Rust编写而成。当我们一次给它大量的输入时，这个标记器可以非常快。

请注意，我们现在在标记函数中省略了padding参数。这是因为在标记的时候将所有样本填充到最大长度的效率不高。一个更好的做法：在构建批处理时填充样本更好，因为这样我们只需要填充到该批处理中的最大长度，而不是整个数据集的最大长度。当输入长度变化很大时，这可以节省大量时间和处理能力!

下面是我们如何在所有数据集上同时应用标记函数。我们在调用map时使用了batch =True，这样函数就可以同时应用到数据集的多个元素上，而不是分别应用到每个元素上。这将使我们的预处理快许多

In [41]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [31]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

在使用预处理函数map()时，甚至可以通过传递num_proc参数使用并行处理。我们在这里没有这样做，因为🤗标记器库已经使用多个线程来更快地标记我们的样本，但是如果您没有使用该库支持的快速标记器，使用num_proc可能会加快预处理。

我们的标记函数(tokenize_function)返回包含输入词id(input_ids) ， 注意力遮罩(attention_mask) 和 类型标记ID(token_type_ids) 键的字典,所以这三个字段被添加到数据集的标记的结果中。注意，如果预处理函数map()为现有键返回一个新值，那将会修改原有键的值。

最后一件我们需要做的事情是，当我们一起批处理元素时，将所有示例填充到最长元素的长度——我们称之为动态填充。

### __四、动态填充__

负责在批处理中将数据整理为一个batch的函数称为collate函数。它是你可以在构建DataLoader时传递的一个参数，默认是一个函数，它将把你的数据集转换为PyTorch张量，并将它们拼接起来(如果你的元素是列表、元组或字典，则会使用递归)。这在我们的这个例子中下是不可行的，因为我们的输入不是都是相同大小的。我们故意在之后每个batch上进行填充，避免有太多填充的过长的输入。这将大大加快训练速度，但请注意，如果你在TPU上训练，这可能会导致问题——TPU喜欢固定的形状，即使这需要额外的填充。

为了解决句子长度统一的问题，我们必须定义一个collate函数，该函数会将每个batch句子填充到正确的长度。幸运的是，🤗transformer库通过DataCollatorWithPadding为我们提供了这样一个函数。当你实例化它时，需要一个标记器(用来知道使用哪个词来填充，以及模型期望填充在左边还是右边)，并将做你需要的一切:

In [38]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

为了测试这个新玩具，让我们从我们的训练集中抽取几个样本。这里，我们删除列idx, sentence1和sentence2，因为不需要它们，并查看一个batch中每个条目的长度:

In [39]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

毫无疑问，我们得到了不同长度的样本，从32到67。动态填充意味着该批中的所有样本都应该填充到长度为67，这是该批中的最大长度。如果没有动态填充，所有的样本都必须填充到整个数据集中的最大长度，或者模型可以接受的最大长度。让我们再次检查data_collator是否正确地动态填充了这批样本：

In [40]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

### __使用 Trainer API 微调模型__

🤗 Transformers提供了一个 Trainer 类来帮助您在自己的数据集上微调任何预训练模型。完成上一节中的所有数据预处理工作后，您只需要执行几个步骤来创建 Trainer .最难的部分可能是为 Trainer.train()配置运行环境，因为它在 CPU 上运行速度会非常慢。如果您没有设置 GPU，您可以访问免费的 GPU 或 TPUGoogle Colab.

下面的示例假设您已经执行了上一节中的示例。下面这段代码，概括了您需要提前运行的代码：

In [42]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [49]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [55]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

__Training:__

在我们定义我们的 Trainer 之前首先要定义一个 TrainingArguments 类，它将包含 Trainer用于训练和评估的所有超参数。您唯一必须提供的参数是保存训练模型的目录，以及训练过程中的检查点。对于其余的参数，您可以保留默认值，这对于基本微调应该非常有效。

In [45]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

第二步是定义我们的模型。正如在[之前的章节](/2_Using Transformers/Introduction)一样，我们将使用 AutoModelForSequenceClassification 类，它有两个参数：

In [46]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

你会注意到，和第二章不一样的是，在实例化此预训练模型后会收到警告。这是因为 BERT 没有在句子对分类方面进行过预训练，所以预训练模型的头部已经被丢弃，而是添加了一个适合句子序列分类的新头部。警告表明一些权重没有使用（对应于丢弃的预训练头的那些），而其他一些权重被随机初始化（新头的那些）。最后鼓励您训练模型，这正是我们现在要做的。

一旦我们有了我们的模型，我们就可以定义一个 Trainer 通过将之前构造的所有对象传递给它——我们的model 、training_args ，训练和验证数据集，data_collator ，和 tokenizer ：

In [48]:
tokenized_datasets["train"]

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [50]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

请注意，当您在这里完成tokenizer后，默认 Trainer使用 的data_collator会使用之前预定义的 DataCollatorWithPadding ，因此您可以在这个例子中跳过 data_collator=data_collator。在第 2 节中向您展示这部分处理仍然很重要！

为了让预训练模型在在我们的数据集上微调，我们只需要调用Trainer的train() 方法 ：

In [52]:
trainer.train()

这将开始微调（在GPU上应该需要几分钟），并每500步报告一次训练损失。但是，它不会告诉您模型的性能如何（或质量如何）。这是因为:

我们没有通过将evaluation_strategy设置为“steps”(在每次更新参数的时候评估)或“epoch”(在每个epoch结束时评估)来告诉Trainer在训练期间进行评估。
我们没有为Trainer提供一个compute_metrics()函数来直接计算模型的好坏(否则评估将只输出loss，这不是一个非常直观的数字)。

__评估__

让我们看看如何构建一个有用的 compute_metrics() 函数并在我们下次训练时使用它。该函数必须采用 EvalPrediction 对象（带有 predictions 和 label_ids 字段的参数元组）并将返回一个字符串到浮点数的字典（字符串是返回的指标的名称，而浮点数是它们的值）。我们可以使用 Trainer.predict() 命令来使用我们的模型进行预测：

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

predict() 的输出结果是具有三个字段的命名元组： predictions , label_ids ， 和 metrics .这 metrics 字段将只包含传递的数据集的loss，以及一些运行时间（预测所需的总时间和平均时间）。如果我们定义了自己的 compute_metrics() 函数并将其传递给 Trainer ，该字段还将包含compute_metrics()的结果。

predict() 方法是具有三个字段的命名元组： predictions , label_ids ， 和 metrics .这 metrics 字段将只包含传递的数据集的loss，以及一些运行时间（预测所需的总时间和平均时间）。如果我们定义了自己的 compute_metrics() 函数并将其传递给 Trainer ，该字段还将包含compute_metrics() 的结果。如你看到的， predictions 是一个形状为 408 x 2 的二维数组（408 是我们使用的数据集中元素的数量）。这些是我们传递给predict()的数据集的每个元素的结果(logits)（正如你在之前的章节看到的情况）。要将我们的预测的可以与真正的标签进行比较，我们需要在第二个轴上取最大值的索引：

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

现在建立我们的 compute_metric() 函数来较为直观地评估模型的好坏，我们将使用 🤗 Evaluate 库中的指标。我们可以像加载数据集一样轻松加载与 MRPC 数据集关联的指标，这次使用 evaluate.load() 函数。返回的对象有一个 compute()方法我们可以用来进行度量计算的方法：

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

您获得的确切结果可能会有所不同，因为模型头的随机初始化可能会影响最终建立的模型。在这里，我们可以看到我们的模型在验证集上的准确率为 85.78%，F1 分数为 89.97。这是用于评估 GLUE 基准的 MRPC 数据集结果的两个指标。而在BERT 论文中展示的基础模型的 F1 分数为 88.9。那是 uncased 模型，而我们目前正在使用 cased 模型，通过改进得到了更好的结果。

最后将所有东西打包在一起，我们得到了我们的 compute_metrics() 函数：

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

为了查看模型在每个训练周期结束的好坏，下面是我们如何使用compute_metrics()函数定义一个新的 Trainer ：

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

请注意，我们设置了了一个新的 TrainingArguments 它的evaluation_strategy 设置为 epoch 并创建了一个新模型。如果不创建新的模型就直接训练，就只会继续训练之前我们已经训练过的模型。要启动新的训练运行，我们执行：

In [ ]:
trainer.train()

这一次，它将在训练loss之外，还会输出每个 epoch 结束时的验证loss和指标。同样，由于模型的随机头部初始化，您达到的准确率/F1 分数可能与我们发现的略有不同，但它应该在同一范围内。

这 Trainer 将在多个 GPU 或 TPU 上开箱即用，并提供许多选项，例如混合精度训练（在训练的参数中使用 fp16 = True ）。我们将在第 10 章讨论它支持的所有内容。

使用Trainer API微调的介绍到此结束。对最常见的 NLP 任务执行此操作的示例将在第 7 章中给出，但现在让我们看看如何在纯 PyTorch 中执行相同的操作。